# Notebook 00: Setup and Data Check

This notebook verifies that the environment is set up correctly and all data sources
are accessible. Run this once before working with the other notebooks.

**Data sources tested:**
1. SWIID (Standardized World Income Inequality Database)
2. WID (World Inequality Database)
3. OECD IDD (Income Distribution Database)
4. Eurostat EU-SILC
5. US Census CPS Historical Tables

In [ ]:
import sys
sys.path.insert(0, '..')

# Verify core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from data.config import FOCUS_COUNTRIES, COUNTRY_NAMES, PROJECT_ROOT
from data.plotting import set_style

set_style()
print(f"Project root: {PROJECT_ROOT}")
print(f"Focus countries: {[COUNTRY_NAMES[c] for c in FOCUS_COUNTRIES]}")
print("Core imports OK.")

## 1. SWIID — Standardized World Income Inequality Database

The SWIID (Frederick Solt) maximizes comparability of Gini coefficients across
199 countries from 1960 to the present. It provides both **market income** (pre-tax,
pre-transfer) and **disposable income** (post-tax, post-transfer) Gini coefficients.

This is our backbone for cross-country Gini comparisons.

In [ ]:
from data import swiid

# Load full dataset
swiid_full = swiid.load_swiid()
print(f"SWIID: {len(swiid_full)} rows, {swiid_full['country'].nunique()} countries")
print(f"Years: {swiid_full['year'].min()} - {swiid_full['year'].max()}")
print(f"Columns: {list(swiid_full.columns)}")

# Test filtered query
gini_test = swiid.get_gini(FOCUS_COUNTRIES, gini_type='disp', start_year=2015, end_year=2020)
print(f"\nFiltered query: {len(gini_test)} rows for {gini_test['country_code'].nunique()} countries")
gini_test.groupby('country_code')['gini'].last()

## 2. WID — World Inequality Database

The WID uses the Distributional National Accounts (DINA) methodology to produce
inequality estimates consistent with national accounts totals. It provides pre-tax
and post-tax income shares, wealth shares, and average income at each percentile.

**Note:** First run downloads per-country CSV files (~5-15 MB each).

In [ ]:
from data import wid

# Test: pre-tax top 10% income share for focus countries
top10 = wid.get_income_shares(
    FOCUS_COUNTRIES,
    income_concept='pretax',
    percentiles=['p90p100'],
    start_year=2015, end_year=2020,
)
print(f"WID income shares: {len(top10)} rows")
print("\nMost recent top 10% pre-tax income share:")
top10.sort_values('year').groupby('country').last()[['year', 'value']]

In [ ]:
# Test: wealth shares
wealth = wid.get_wealth_shares(
    FOCUS_COUNTRIES,
    percentiles=['p90p100', 'p99p100'],
    start_year=2015, end_year=2020,
)
print(f"WID wealth shares: {len(wealth)} rows")
if not wealth.empty:
    print("\nMost recent top 10% and top 1% wealth shares:")
    display(wealth.sort_values('year').groupby(['country', 'percentile']).last()[['year', 'value']])

## 3. OECD Income Distribution Database

The OECD IDD provides standardized Gini coefficients and income shares from
national statistical offices. The `pandasdmx` package queries their SDMX API.

**Note:** The OECD API can be slow or flaky. If this cell fails, the SWIID and WID
data loaders provide sufficient coverage for most analyses.

In [ ]:
from data import oecd

try:
    oecd_data = oecd.get_idd_gini(FOCUS_COUNTRIES)
    print(f"OECD IDD: {len(oecd_data)} rows")
    if not oecd_data.empty:
        print(f"Columns: {list(oecd_data.columns)}")
        display(oecd_data.head())
    else:
        print("OECD query returned empty. This is common — the API can be unreliable.")
        print("SWIID provides equivalent Gini data as a fallback.")
except Exception as e:
    print(f"OECD query failed: {e}")
    print("This is non-critical — SWIID provides equivalent data.")

## 4. Eurostat EU-SILC

Eurostat provides inequality data for all EU member states plus Norway, Switzerland,
and Iceland, based on the EU-SILC household survey. Coverage starts in 2004-2005.

In [ ]:
from data import eurostat_loader

try:
    es_gini = eurostat_loader.get_gini(start_year=2015, end_year=2022)
    print(f"Eurostat Gini: {len(es_gini)} rows")
    if not es_gini.empty:
        print(f"Countries: {es_gini['country'].nunique()}")
        display(es_gini.head(10))
    else:
        print("Eurostat query returned empty.")
except Exception as e:
    print(f"Eurostat query failed: {e}")
    print("This is non-critical for the core analysis.")

## 5. US Census CPS Historical Tables

The Census Bureau publishes the official US Gini index and income by quintile
from 1967 to the present, based on the Current Population Survey (CPS).

In [ ]:
from data import census

try:
    us_gini = census.get_us_gini_timeseries()
    print(f"US Census Gini: {len(us_gini)} years")
    if not us_gini.empty:
        print(f"Range: {us_gini['year'].min()} - {us_gini['year'].max()}")
        print(f"Latest Gini: {us_gini.iloc[-1]['gini']:.4f} ({int(us_gini.iloc[-1]['year'])})")
    else:
        print("Census data returned empty. Check if the URL has changed.")
except Exception as e:
    print(f"Census download failed: {e}")
    print("You can download manually from:")
    print("https://www.census.gov/data/tables/time-series/demo/income-poverty/historical-income-inequality.html")

## Summary

The table below shows which data sources are available and working.

In [ ]:
summary = {
    'Source': ['SWIID', 'WID', 'OECD IDD', 'Eurostat', 'US Census'],
    'Status': [],
    'Coverage': [],
}

# Check each source
try:
    s = swiid.load_swiid()
    summary['Status'].append('OK')
    summary['Coverage'].append(f"{s['country'].nunique()} countries, {s['year'].min()}-{s['year'].max()}")
except:
    summary['Status'].append('FAILED'); summary['Coverage'].append('-')

try:
    w = wid.get_income_shares(['US'], percentiles=['p90p100'], start_year=2019, end_year=2020)
    summary['Status'].append('OK' if not w.empty else 'EMPTY')
    summary['Coverage'].append(f"Per-country CSV downloads")
except:
    summary['Status'].append('FAILED'); summary['Coverage'].append('-')

try:
    o = oecd.get_idd_gini(['US'])
    summary['Status'].append('OK' if not o.empty else 'EMPTY/UNRELIABLE')
    summary['Coverage'].append('OECD members')
except:
    summary['Status'].append('FAILED'); summary['Coverage'].append('-')

try:
    e = eurostat_loader.get_gini(start_year=2019, end_year=2022)
    summary['Status'].append('OK' if not e.empty else 'EMPTY')
    summary['Coverage'].append('EU + EFTA')
except:
    summary['Status'].append('FAILED'); summary['Coverage'].append('-')

try:
    c = census.get_us_gini_timeseries()
    summary['Status'].append('OK' if not c.empty else 'EMPTY')
    summary['Coverage'].append(f"US, {c['year'].min():.0f}-{c['year'].max():.0f}" if not c.empty else '-')
except:
    summary['Status'].append('FAILED'); summary['Coverage'].append('-')

pd.DataFrame(summary)